In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import (LogisticRegression, Perceptron,
                                  SGDClassifier)
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [3]:
train_data = pd.read_csv("data/train.tsv", sep="\t")
test_data = pd.read_csv("data/test.tsv", sep="\t")
sample = pd.read_csv("data/sample_submit.csv")

In [10]:
train_data.head(10)

,id,Tournament,Sex,Year,Player1,Player2,Round,Result,FNL.1,FNL.2,...,BPC.2,BPW.2,NPA.2,NPW.2,TPW.2,ST1.2,ST2.2,ST3.2,ST4.2,ST5.2
0,0,FrenchOpen,men,2013,Jarkko Nieminen,Paul-Henri Mathieu,1,1,3,2,...,5,15,24.0,33.0,163.0,4,6.0,6.0,6.0,2.0
1,3,AusOpen,women,2013,Varvara Lepchenko,Simona Halep,2,0,1,2,...,6,15,1.0,2.0,90.0,4,6.0,6.0,NaN,NaN
2,4,Wimbledon,women,2013,H.Watson,M.Keys,1,0,0,2,...,7,5,8.0,5.0,NaN,6,7.0,NaN,NaN,NaN
3,7,USOpen,women,2013,S Williams,F Schiavone,1,1,2,0,...,0,0,6.0,3.0,NaN,0,1.0,NaN,NaN,NaN
4,9,AusOpen,women,2013,Tadeja Majeric,Ajla Tomljanovic,1,0,1,2,...,3,6,NaN,NaN,97.0,3,7.0,6.0,NaN,NaN
5,11,USOpen,women,2013,C Giorgi,R Vinci,4,0,0,2,...,8,4,5.0,4.0,NaN,6,6.0,NaN,NaN,NaN
6,13,USOpen,women,2013,L Robson,C Garcia,2,1,2,0,...,2,2,10.0,7.0,NaN,4,6.0,NaN,NaN,NaN
7,16,FrenchOpen,men,2013,Lukas Lacko,Sam Querrey,1,0,0,3,...,4,7,12.0,13.0,93.0,6,6.0,6.0,NaN,NaN
8,19,USOpen,men,2013,Novak Djokovic,Mikhail Youzhny,1,1,3,1,...,2,10,10.0,21.0,87.0,3,2.0,6.0,0.0,NaN
9,23,AusOpen,men,2013,Tomas Berdych,David Ferrer,5,1,3,1,...,3,12,12.0,13.0,119.0,1,4.0,6.0,4.0,NaN


In [5]:
test_data.head()

,id,Tournament,Sex,Year,Player1,Player2,Round,FNL.1,FNL.2,FSP.1,...,BPC.2,BPW.2,NPA.2,NPW.2,TPW.2,ST1.2,ST2.2,ST3.2,ST4.2,ST5.2
0,1,Wimbledon,women,2013,A.Radwanska,M.Keys,3,2.0,1.0,71,...,10.0,2.0,34.0,22.0,NaN,5.0,6.0,3.0,NaN,NaN
1,2,AusOpen,men,2013,Nick Kyrgios,Benjamin Becker,1,3.0,1.0,54,...,3.0,7.0,6.0,6.0,114.0,3.0,7.0,2.0,6.0,NaN
2,5,Wimbledon,men,2013,V.Pospisil,M.Youzhny,2,2.0,3.0,63,...,7.0,4.0,46.0,32.0,NaN,6.0,6.0,7.0,3.0,6.0
3,6,FrenchOpen,women,2013,Caroline Wozniacki,Bojana Jovanovski,2,0.0,2.0,70,...,3.0,7.0,2.0,5.0,72.0,7.0,6.0,NaN,NaN,NaN
4,8,Wimbledon,men,2013,N.Djokovic,J.Chardy,3,3.0,0.0,74,...,0.0,0.0,23.0,12.0,NaN,3.0,2.0,2.0,NaN,NaN


In [6]:
sample.head()

,1,0
0,2,1
1,5,0
2,6,0
3,8,1
4,10,0


In [8]:
full_data = [train_data, test_data]

In [15]:
train_data.info()
print("_"*40)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 46 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          471 non-null    int64  
 1   Tournament  471 non-null    object 
 2   Sex         471 non-null    object 
 3   Year        471 non-null    int64  
 4   Player1     471 non-null    object 
 5   Player2     471 non-null    object 
 6   Round       471 non-null    int64  
 7   Result      471 non-null    int64  
 8   FNL.1       471 non-null    int64  
 9   FNL.2       471 non-null    int64  
 10  FSP.1       471 non-null    int64  
 11  FSW.1       471 non-null    int64  
 12  SSP.1       471 non-null    int64  
 13  SSW.1       471 non-null    int64  
 14  ACE.1       462 non-null    float64
 15  DBF.1       468 non-null    float64
 16  WNR.1       408 non-null    float64
 17  UFE.1       408 non-null    float64
 18  BPC.1       471 non-null    int64  
 19  BPW.1       471 non-null    i

In [22]:
#第4,5セットの獲得ゲーム数ST4,ST5は欠損値が多いため削除
drop_columns = ["ST4.1", "ST5.1", "ST4.2", "ST5.2"]

train_data = train_data.drop(drop_columns, axis=1)
test_data = test_data.drop(drop_columns, axis=1)